Brochure Generator with UI (using Gradio)
Requirement :
- user can pick model : claude, gpt, or gemini
- user need to fill company name, company page url, brochure language
- user can pick brochure theme : formal, informal, semi-formal, funny.

In [103]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai
import json
import requests
from bs4 import BeautifulSoup
import gradio as gr # oh yeah!

In [104]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBG


In [105]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [106]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [107]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [108]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [135]:
def stream_gemini(prompt):
    # Combine the system message and user prompt
    conversation = f"{system_message}\nUser: {prompt}"

        # Call Gemini's API with the constructed conversation
    gemini = google.generativeai.GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=system_message
    )
    response = gemini.generate_content(prompt)

    # Simulate streaming by yielding parts of the response progressively
    result = response.text
    partial_result = ""
    for char in result:
        partial_result += char
        yield partial_result


In [132]:
def stream_brochure(company_name, url, model, language, tone):
    prompt = f"In {language}, Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    prompt += f"\n\n you should generate this brochure with this {tone} of emotion and personality"
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [137]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model"),
        gr.Dropdown(["English", "Bahasa Indonesia", "Spanish"], label="Select langauge"),
        gr.Dropdown(["Formal", "Informal", "Funny", "Sarcastic"], label="Select tone")
    ],
    
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7908
* Running on public URL: https://b327873332282865d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
